# Note

The Machine Learning methods presented here are only some of the ways of approaching the portfolio optimization problem. There are multiple ways of incoporating ML in the financial models and it is up to you to come up with more interesting and appropriate approaches. We have tried to use multiple ML models in different sections as an example.

The general rule of thumb in the approaches is as follows:
- Use non-linear regression for predicting future values of stocks
- Use appropriate linear regression for financial models (Single Index, CAPM) according to their standard formulas

# Load your data

We present here a basic way of importing stock data which will be used in the subsequent sections.

In [2]:
import yfinance as yf
import datetime
# Define the ticker symbols for the market index and individual assets
market_index_ticker = "^GSPC"  # S&P 500 index
google_ticker = "GOOGL"  # Google
apple_ticker = "AAPL"  # Apple

# Define the start and end dates for the historical data
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=5*365)

# Fetch the historical data using yfinance
market_index_data = yf.download(
    market_index_ticker, start=start_date, end=end_date)
google_data = yf.download(google_ticker, start=start_date, end=end_date)
apple_data = yf.download(apple_ticker, start=start_date, end=end_date)

# Print the fetched data
print("Market Index Data:")
print(market_index_data.head())
print("\nGoogle Data:")
print(google_data.head())
print("\nApple Data:")
print(apple_data.head())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Market Index Data:
                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2018-06-28  2698.689941  2724.340088  2691.989990  2716.310059  2716.310059   
2018-06-29  2727.129883  2743.260010  2718.030029  2718.370117  2718.370117   
2018-07-02  2704.949951  2727.260010  2698.949951  2726.709961  2726.709961   
2018-07-03  2733.270020  2736.580078  2711.159912  2713.219971  2713.219971   
2018-07-05  2724.189941  2737.830078  2716.020020  2736.610107  2736.610107   

                Volume  
Date                    
2018-06-28  3461100000  
2018-06-29  3586800000  
2018-07-02  3095040000  
2018-07-03  1911460000  
2018-07-05  2980160000  

Google Data:
                 Open       Hig

# Mean Variance Optimization

- Load the stock prices for your selected stocks
- Divide data into independent variable X (10 consecutive days of stock values), and dependent variable y (stock value at 10th day into future)
- Train non linear regression model to predict stock value 10 day into the future
- Use the 10 future stock values and apply mean-variance optimization using pypfopt module

In [3]:
!pip install PyPortfolioOpt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 3.5 MB/s eta 0:00:00


In [4]:
from pypfopt import plotting
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from pypfopt import expected_returns, risk_models, EfficientFrontier


apple_prices = apple_data['Close'].tolist()
google_prices = google_data['Close'].tolist()

# Dependent variable - 10 consecutive days of stock prices
apple_x = [apple_prices[i:i+10] for i in range(len(apple_prices)-20)]
# Independent variable - stock price 10th day into the future
apple_y = [apple_prices[i+10] for i in range(10,len(apple_prices)-10)]

apple_test = [apple_prices[i:i+10] for i in range(len(apple_prices)-20,len(apple_prices)-10)]

reg = RandomForestRegressor()
reg.fit(apple_x,apple_y)
# Predict stock price for 10 future days
apple = reg.predict(apple_test)

google_x = [google_prices[i:i+10] for i in range(len(google_prices)-20)]
google_y = [google_prices[i+10] for i in range(10,len(google_prices)-10)]

google_test = [google_prices[i:i+10] for i in range(len(google_prices)-20,len(google_prices)-10)]

reg = RandomForestRegressor()
reg.fit(google_x,google_y)
google = reg.predict(google_test)

future_prices = {'apple':apple,'google':google}
future_prices = pd.DataFrame(future_prices)


# Construct covariance matrix of future stock prices
# cov_matrix = risk_models.sample_cov(future_prices)
S = risk_models.CovarianceShrinkage(future_prices).ledoit_wolf()
# plotting.plot_covariance(S, plot_correlation=True)

# Use capm to find expected returns on future prices
mu = expected_returns.capm_return(future_prices)
print(mu)
# Do mean variance optimization using efficient frontier
ef = EfficientFrontier(mu, S)
ef.min_volatility()
weights = ef.clean_weights()
print(weights)
# weights = ef.max_sharpe(risk_free_rate=0.02)
# cleaned_weights = ef.clean_weights()
# print(cleaned_weights)

apple     0.012297
google   -1.329192
Name: mkt, dtype: float64
OrderedDict([('apple', 0.57174), ('google', 0.42826)])


# Single Index Model

- Load the stock prices for your selected stocks
- Divide data into independent variable X (10 consecutive days of market index values), and dependent variable y (market index value at 10th day into future)
- Train non linear regression model to predict market index value 10 day into the future
- Fit separate linear models each for a stock according to formula of single index model
- The index model can
be written as the following regression equation:

 $$R_{i}(t) = \alpha_{i} + \beta_{i}R_{M}(t) + e_{i}(t)$$
 Since, $E(e_{i}) = 0$
 $$E(R_{i}) = \alpha_{i} + \beta_{i}E(R_{M})$$
- Use the return values of the stock(y) and the market(x) and fit a linear regression model
- Compare the coefficients of the fitted line to find $\alpha_{i}$ and $\beta_{i}$
- Use 10 predicted market returns to predict returns of each stock
- Use the Treynor Black model for portfolio optimization
- Step 1: $$w_{i}^{0} = \frac{\alpha_{i}}{\sigma^{2}(e_{i})}$$
- Step 2: $$w_{i} = \frac{w_{i}^{0}}{\sum w_{i}^{0}}$$
- Step 3: $$\alpha_{A} = \sum w_{i}\alpha_{i}$$
- Step 4: $$\sigma^{2}(e_{A}) = \sum w_{i}^{2}\sigma^{2}(e_{i})$$
- Step 5: $$w_{A}^{0} = \frac{\alpha_{A}/\sigma^{2}(e_{A})}{E(R_{m})/\sigma^{2}_{M}}$$
- Step 6: $$\beta_{A} = \sum w_{i}\beta_{i}$$
- Step 7: $$w_{A}^{*} = \frac{w_{A}^{0}}{1+(1-\beta_{A})w_{A}^{0}}$$
- Step 8: $$w_{M}^{*} = 1-w_{A}^{*}$$
$$w_{i}^{*} = w_{A}^{*}w_{i}$$
- Step 9: $$E(R_{P}) = (w_{M}^{*}+w_{A}^{*}\beta_{A})E(R_{M}) +w_{A}^{*}\alpha_{A}$$
- Step 10: $$\sigma_{P}^{2} = (w_{M}^{*}+w_{A}^{*}\beta_{A})^{2}\sigma_{M}^{2} + (w_{A}^{*}\sigma(e_{A}))^{2}$$







In [45]:
from pypfopt import plotting
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from pypfopt import expected_returns, risk_models, EfficientFrontier

# Returns of stocks and market
apple_returns = ((apple_data['Close'] / apple_data['Close'].shift(1))-1).dropna().tolist()
google_returns = ((google_data['Close'] / google_data['Close'].shift(1))-1).dropna().tolist()
market_returns = ((market_index_data['Close'] / market_index_data['Close'].shift(1))-1).dropna().tolist()

# Future marker returns prediction
market_x = [market_returns[i:i+10] for i in range(len(market_returns)-20)]
market_y = [market_returns[i+10] for i in range(10, len(market_returns)-10)]

market_test = [market_returns[i:i+10]
              for i in range(len(market_returns)-20, len(market_returns)-10)]

reg = RandomForestRegressor()
reg.fit(market_x, market_y)
market_future = reg.predict(market_test)

# Fitting single index model regression on market return and stock return
single_index_reg_apple = LinearRegression()
single_index_reg_apple.fit(np.array(market_returns).reshape(-1, 1),y=apple_returns)
apple_future = single_index_reg_apple.predict(np.array(market_future).reshape(-1, 1))


# Beta for apple stock
beta_apple = single_index_reg_apple.coef_[0]
# alpha for apple stock
alpha_apple = single_index_reg_apple.intercept_
print("Beta of Apple: ", beta_apple)
print("Alpha of Apple: ", alpha_apple)

y_pred = single_index_reg_apple.predict(np.array(market_returns).reshape(-1, 1))
residuals = apple_returns - y_pred
residual_variance_apple = np.var(residuals)

# variance of the residuals for apple
print("Residual variance of Apple: ", residual_variance_apple)
print("-------------------------")

single_index_reg_google = LinearRegression()
single_index_reg_google.fit(np.array(market_returns).reshape(-1, 1),google_returns)
google_future = single_index_reg_google.predict(
    np.array(market_future).reshape(-1, 1))

# Beta for google stock
beta_google = single_index_reg_google.coef_[0]
# alpha for google stock
alpha_google = single_index_reg_google.intercept_
print("Beta of Google: ", beta_google)
print("Alpha of Google: ", alpha_google)

y_pred = single_index_reg_google.predict(np.array(market_returns).reshape(-1, 1))
residuals = google_returns - y_pred
residual_variance_google = np.var(residuals)

# variance of the residuals for apple
print("Residual variance of Google: ", residual_variance_google)
print("-------------------------")

# Using Treynor Black model for portfolio optimization

#STEP 1:
# Compute the initial position of each security:
w_apple = alpha_apple/residual_variance_apple
w_google = alpha_google/residual_variance_google
print("Apple Weight:", w_apple)
print("Google Weight:", w_google)

#STEP 2:
# Scale the initial positions:
w_apple_scaled = w_apple/(w_apple+w_google)
w_google_scaled = w_google/(w_apple+w_google)
print("Apple Weight Scaled:", w_apple_scaled)
print("Google Weight Scaled:", w_google_scaled)

#STEP 3:
# Compute the alpha of the active portfolio:
alpha_portfolio = w_apple_scaled*alpha_apple+w_google_scaled*alpha_google
print("Alpha of the active portfolio: ", alpha_portfolio)

#STEP 4:
# Compute the residual variance of active portfolio:
residual_variance_portfolio = w_apple_scaled*w_apple_scaled*residual_variance_apple + w_google_scaled*w_google_scaled*residual_variance_google
print("Residual variance of the active portfolio: ", residual_variance_portfolio)

#STEP 5:
# Compute the initial position in active portfolio:
# Note: The S&P 500’s long-term standard deviation (volatility) is around 12%. Hence, variance of S&P is 0.0114
residual_variance_market = 0.0114
risk_premium_market = 0.056
initial_position_portfolio = (alpha_portfolio*residual_variance_market)/(residual_variance_portfolio*risk_premium_market)
print("Initial position of active portfolio: ", initial_position_portfolio)

#STEP 6:
# Compute the beta of active portfolio:
beta_portfolio = w_apple_scaled*beta_apple+w_google_scaled*beta_google
print("Beta of the active portfolio: ", beta_portfolio)

#STEP 7:
# Adjust the initial position in active portfolio
adjusted_position_portfolio = initial_position_portfolio/1+(1-beta_portfolio)*initial_position_portfolio
print("Adjusted position of the active portfolio: ", adjusted_position_portfolio)

#STEP 8:
# Optimal risky portfolio now has weights:
final_weight_market = 1-adjusted_position_portfolio
final_weight_apple = adjusted_position_portfolio*w_apple_scaled
final_weight_google = adjusted_position_portfolio*w_google_scaled

print("------------------------------")
print("Final Weights: ")
print("Weight Market S&P: ", final_weight_market)
print("Weight Apple: ", final_weight_apple)
print("Weight Apple: ", final_weight_google)
print("------------------------------")


#STEP 9:
# Calculate the risk premium of P (Optimal risky portfolio):
risk_premium_porfolio = (final_weight_market+adjusted_position_portfolio*beta_portfolio)*risk_premium_market + adjusted_position_portfolio*alpha_portfolio
print("Risk premium of portfolio: ", risk_premium_porfolio)

#STEP 10:
# Compute the variance of Portfolio:
portfolio_variance = (final_weight_market+adjusted_position_portfolio*beta_portfolio)*(final_weight_market+adjusted_position_portfolio*beta_portfolio)*residual_variance_market + adjusted_position_portfolio*adjusted_position_portfolio*residual_variance_portfolio
print("Variance of portfolio: ", portfolio_variance)

Beta of Apple:  1.2354278944714905
Alpha of Apple:  0.0007505860876984197
Residual variance of Apple:  0.00015472764341695558
-------------------------
Beta of Google:  1.1422729066956818
Alpha of Google:  0.0002535994303931205
Residual variance of Google:  0.00016061679014244966
-------------------------
Apple Weight: 4.851014796856707
Google Weight: 1.5789098398007166
Apple Weight Scaled: 0.7544434921057632
Google Weight Scaled: 0.24555650789423686
Alpha of the active portfolio:  0.0006285477796605008
Residual variance of the active portfolio:  9.775352203618508e-05
Initial position of active portfolio:  1.3089489438909754
Beta of the active portfolio:  1.2125530809803327
Adjusted position of the active portfolio:  1.030727813020996
------------------------------
Final Weights: 
Weight Market S&P:  -0.03072781302099603
Weight Apple:  0.7776258906660964
Weight Apple:  0.2531019223548997
------------------------------
Risk premium of portfolio:  0.06891658652765371
Variance of portfoli

# CAPM

Similar to Single Index Model.

Here the Regression equation is:

$$R_{i} = R_{f} + β_{i} (R_{m} - R_{f})$$

We will fit a linear regression for this equation to find $\beta_{i}$
$$R_{i} - R_{f} = β_{i} (R_{m} - R_{f}) + \text{error}$$

Here we are assuming the risk free rate of return($R_{f}$) as 0.01 - T-bill rate

In [48]:
from pypfopt import expected_returns
from pypfopt import plotting
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from pypfopt import expected_returns, risk_models, EfficientFrontier

# T-bill rate
risk_free_rate = 0.01

apple_returns = (
    (apple_data['Close'] / apple_data['Close'].shift(1))-1-risk_free_rate).dropna().tolist()
google_returns = (
    (google_data['Close'] / google_data['Close'].shift(1))-1-risk_free_rate).dropna().tolist()
market_returns = (
    (market_index_data['Close'] / market_index_data['Close'].shift(1))-1-risk_free_rate).dropna().tolist()

# apple_prices = apple_data['Close'].tolist()
# google_prices = google_data['Close'].tolist()
market_x = [market_returns[i:i+10] for i in range(len(market_returns)-20)]
market_y = [market_returns[i+10] for i in range(10, len(market_returns)-10)]

market_test = [market_returns[i:i+10]
               for i in range(len(market_returns)-20, len(market_returns)-10)]

reg = MLPRegressor(hidden_layer_sizes=(100,100))
reg.fit(market_x, market_y)
market_future = reg.predict(market_test)

# print(market_future)

single_index_reg = Ridge()
single_index_reg.fit(np.array(market_returns).reshape(-1, 1), y=apple_returns)
apple_future = single_index_reg.predict(np.array(market_future).reshape(-1, 1))

single_index_reg = Ridge()
single_index_reg.fit(np.array(market_returns).reshape(-1, 1), google_returns)
google_future = single_index_reg.predict(
    np.array(market_future).reshape(-1, 1))

future_returns = {'apple': apple, 'google': google}
future_returns = pd.DataFrame(future_returns)


S = risk_models.sample_cov(future_returns)
# S = risk_models.CovarianceShrinkage(future_returns).ledoit_wolf()
# plotting.plot_covariance(S, plot_correlation=True)
# You don't have to provide expected returns in this case


ef = EfficientFrontier(None, S)
ef.min_volatility()
weights = ef.clean_weights()
print(weights)
# weights = ef.max_sharpe(risk_free_rate=-0.2)
# cleaned_weights = ef.clean_weights()
# print(cleaned_weights)
# print(mu)


OrderedDict([('apple', 0.8449), ('google', 0.1551)])


# Multifactor model

- Load the market factors affecting your stocks (here we have selected 3 random factors) : S&P 500(F1), Crude Oil(F2), and 20+ Year Treasury Bond ETFs(F3)
- Divide data into independent variable X (10 consecutive days of factor values), and dependent variable y (factor value at 10th day into future)
- Train non linear regression model to predict market index value 10 day into the future
- Fit separate linear models each for a stock according to formula of multi factor model
- Here the linear equation is:
$$r = E(r) + \beta_{1}F_{1} + \beta_{2}F_{2} + \beta_{3}F_{3} + e$$
E(r) = expected return on the security \\
$F_{i}$ = the i-th factor \\
$\beta_{i}$ = the security’s sensitivity to movements in the i-th factor \\
e = the idiosyncratic component of the security’s return
- Find $\beta_{i}$ from the fitted line to make the future predictions
- Use 10 predicted factor returns to predict returns of each stock
- Use the future returns of stocks to get stock weights using Mean Variance model

In [62]:
from pypfopt import expected_returns
from pypfopt import plotting
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from pypfopt import expected_returns, risk_models, EfficientFrontier

# Example tickers for S&P 500, Crude Oil, and 20+ Year Treasury Bond ETFs
factor_tickers = ['SPY', 'USO', 'TLT']

# Fetch historical data for the macroeconomic factors
factor_data = yf.download(
    factor_tickers, start=start_date, end=end_date, progress=False)
factor_data = factor_data['Adj Close']

# Calculate the returns for the macroeconomic factors
factor_returns = factor_data.pct_change().dropna()
# print(factor_returns)
spy_returns = factor_returns['SPY']
uso_returns = factor_returns['USO']
tlt_returns = factor_returns['TLT']

# Train ML model to predict future price of macroeconomic factors
spy_x = [spy_returns[i:i+10] for i in range(len(spy_returns)-20)]
spy_y = [spy_returns[i+10] for i in range(10, len(spy_returns)-10)]

# S&P 500
spy_test = [spy_returns[i:i+10]
               for i in range(len(spy_returns)-20, len(spy_returns)-10)]
reg = MLPRegressor(hidden_layer_sizes=(100, 100))
reg.fit(spy_x, spy_y)
spy_future = reg.predict(spy_test)

# Crude Oil
uso_x = [uso_returns[i:i+10] for i in range(len(uso_returns)-20)]
uso_y = [uso_returns[i+10] for i in range(10, len(uso_returns)-10)]

uso_test = [uso_returns[i:i+10]
               for i in range(len(uso_returns)-20, len(uso_returns)-10)]
reg = MLPRegressor(hidden_layer_sizes=(100, 100))
reg.fit(uso_x, uso_y)
uso_future = reg.predict(uso_test)

# Treasury Bond ETF
tlt_x = [tlt_returns[i:i+10] for i in range(len(tlt_returns)-20)]
tlt_y = [tlt_returns[i+10] for i in range(10, len(tlt_returns)-10)]

tlt_test = [tlt_returns[i:i+10]
               for i in range(len(tlt_returns)-20, len(tlt_returns)-10)]

reg = MLPRegressor(hidden_layer_sizes=(100, 100))
reg.fit(tlt_x, tlt_y)
tlt_future = reg.predict(tlt_test)

future_factors = {'SPY': spy_future, 'TLT': tlt_future, 'USO': uso_future}
future_factors = pd.DataFrame(future_factors)
# print(future_factors)

# Fit Multi Factor Linear Regression model using Macroeconomic factors as X and stock returns as y
apple_returns = (
    (apple_data['Close'] / apple_data['Close'].shift(1))-1).dropna().tolist()
google_returns = (
    (google_data['Close'] / google_data['Close'].shift(1))-1).dropna().tolist()

single_index_reg = Ridge()
single_index_reg.fit(factor_returns, apple_returns)
apple_future = single_index_reg.predict(future_factors)

single_index_reg = Ridge()
single_index_reg.fit(factor_returns, google_returns)
google_future = single_index_reg.predict(
    future_factors)

future_returns = {'apple': apple, 'google': google}
# print(list(zip(apple,google)))
future_returns = pd.DataFrame(future_returns)
print(future_returns)

S = risk_models.sample_cov(future_returns)
print(S)
# S = risk_models.CovarianceShrinkage(future_returns).ledoit_wolf()
# plotting.plot_covariance(S, plot_correlation=True)
# You don't have to provide expected returns in this case

print(future_returns.mean())
ef = EfficientFrontier(future_returns.mean(), S)
# ef.min_volatility()
# weights = ef.clean_weights()
# print(weights)
weights = ef.max_sharpe(risk_free_rate=0.0)
cleaned_weights = ef.clean_weights()
print(cleaned_weights)


      apple    google
0  0.002977 -0.000490
1  0.003910  0.002905
2  0.002646  0.003924
3 -0.002167  0.000707
4  0.006790  0.003138
5  0.001383  0.005744
6 -0.001533  0.001839
7  0.002622  0.000796
8  0.007469  0.002868
9  0.002788 -0.000604
             apple       google
apple   784.914239  -332.261227
google -332.261227  2181.908199
apple     0.002689
google    0.002083
dtype: float64
OrderedDict([('apple', 0.72177), ('google', 0.27823)])


# Equity valuation model - Dividend Model

Equity valuation models are used to estimate the intrinsic value of a company's stock

We are using the Dividend Discount Model (DDM):
The Dividend Discount Model values a stock by calculating the present value of its future expected dividends

The formula for the DDM is:

$V = \frac{D_{1}}{(r - g)}$

Where:

V is the intrinsic value of the stock \\
D_{i} is the expected dividend per share in the next period \\
r is the required rate of return or the discount rate \\
g is the expected growth rate of dividends

- Load dividend values of your stock
- Identify the period of change and only keep unique values in a period
- Train non linear regression model to predict next dividend value
- Calculate average dividend growth rate of your stock
- Assume a discount rate for the stock
- Use ML model to predict next dividend value
- Use the predicted dividend value, discount rate and average dividend growth rate to find stock price

**Note:** Google dividend prices were not available from yahoo, so we have assumed an equity value for Google to proceed with portfolio optimization using Mean variance model.

Based on the results of the instrinsic value of stock from the equity valuation model and by comparing it with the current stock price, we can decide to whether buy or sell the stocks. Using these intrinsic values, weights or constraints can be added in the Mean variance model.

In [63]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# get dividends of the stock
apple_stock = yf.Ticker(apple_ticker)
apple_dividends = apple_stock.dividends.loc[start_date:end_date]

# google_stock = yf.Ticker(google_ticker)
# google_dividends = google_stock.dividends.loc[start_date:end_date]
# print(apple_dividends)
# Combine the dividend data into a single DataFrame
dividends = pd.DataFrame({'AAPL':apple_dividends*4})
# print(dividends)
# Calculate the dividend growth rates
dividend_growth_rates = dividends.pct_change().dropna()
# Since we only have dividend changes every 4 months, we remove the 0 values and get the average dividend growth rate which we consider as perpetual growth rate
avg_dividends_growth_rate = dividend_growth_rates.loc[~(
    dividend_growth_rates == 0).all(axis=1)].values.mean()
print(avg_dividends_growth_rate)

# Define the features and target variable for the machine learning model
X = np.unique(dividends.values)[:-1]  # Independent variables (dividend value for the past period)
y = np.unique(dividends.values)[1:]  # Dependent variable (dividend value for the next period)

# print(y)
# Train a machine learning model using regression to predict the next dividend value
model = RandomForestRegressor()
model.fit(X.reshape(-1,1), y)

# Use the trained model to predict the dividend value for the next period
predicted_dividend = model.predict(dividends.values[-1].reshape(1, -1))
next_period_dividend = pd.Series(predicted_dividend.squeeze(), index=dividends.columns)

print(next_period_dividend)
# Apply the DDM formula to calculate the intrinsic value of the stocks
discount_rate = 0.08  # Example discount rate
apple_stock_price = next_period_dividend['AAPL'] / \
    (discount_rate - avg_dividends_growth_rate)
# google_stock_price = google_dividends[-1] / (discount_rate - next_period_growth_rates['GOOGL'])

# Assumption
google_stock_price = 90

print("Intrinsic Value of Apple Stock:", apple_stock_price)
print("Intrinsic Value of Google Stock(assumption):", google_stock_price)

# Filling same value as we only have one predicted value
future_prices = {'apple': [apple_stock_price]*3, 'google': [google_stock_price]*3}

# Using the current stock prices to get an accurate covariance matrix
current_prices = {'apple': apple_data['Close'][-10:], 'google': google_data['Close'][-10:]}
current_prices = pd.DataFrame(current_prices)

# print(list(zip(apple,google)))
future_prices_df = pd.DataFrame(future_prices)
print(current_prices)

# Using the results in Mean Variance model

# Construct covariance matrix of future stock prices
# S = risk_models.sample_cov(current_prices)
# print(S)

# Use capm to find expected returns on future prices
print(future_prices_df)
mu = expected_returns.capm_return(future_prices_df)
print(mu)
# Do mean variance optimization using efficient frontier
ef = EfficientFrontier(mu, S)

# relative_values = [apple_stock_price, 90]  # Intrinsic values
# assign weights or preferences to the stocks based on their valuation outputs

ef.min_volatility()
weights = ef.clean_weights()
print(weights)


0.05636662470288756
AAPL    0.9394
dtype: float64
Intrinsic Value of Apple Stock: 39.74887159324958
Intrinsic Value of Google Stock(assumption): 90
                 apple      google
Date                              
2023-06-12  183.789993  123.639999
2023-06-13  183.309998  123.830002
2023-06-14  183.949997  123.669998
2023-06-15  186.009995  125.089996
2023-06-16  184.919998  123.529999
2023-06-20  185.009995  123.099998
2023-06-21  183.960007  120.550003
2023-06-22  187.000000  123.150002
2023-06-23  186.679993  122.339996
2023-06-26  185.270004  118.339996
       apple  google
0  39.748872      90
1  39.748872      90
2  39.748872      90
apple    NaN
google   NaN
Name: mkt, dtype: float64
OrderedDict([('apple', 0.69235), ('google', 0.30765)])


<ipython-input-63-0e1dfff84dea>:8: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  apple_dividends = apple_stock.dividends.loc[start_date:end_date]


In [69]:
test = {'apple': [apple_stock_price]*10, 'google': [google_stock_price]*10}
test = pd.DataFrame(test)
mu = expected_returns.mean_historical_return(test)
print(mu)

apple     0.0
google    0.0
dtype: float64


In [74]:
print(dividends.values[-10].reshape(1, -1))
print(dividends[-10:])

[[0.82]]
                           AAPL
Date                           
2021-02-05 00:00:00-05:00  0.82
2021-05-07 00:00:00-04:00  0.88
2021-08-06 00:00:00-04:00  0.88
2021-11-05 00:00:00-04:00  0.88
2022-02-04 00:00:00-05:00  0.88
2022-05-06 00:00:00-04:00  0.92
2022-08-05 00:00:00-04:00  0.92
2022-11-04 00:00:00-04:00  0.92
2023-02-10 00:00:00-05:00  0.92
2023-05-12 00:00:00-04:00  0.96


# Black litterman model

In this model, we are using Machine learning to forecast the returns and using that to fill the subjective views(confidence)

- Use non linear ML model to predict stock prices in future (same as we did in mean-variance section)
- Use the predicted future return of stocks as you "views" for black litterman model
- Assign confidence using accuracy of your ML model or arbitrarily based on your market understanding
- Fit the Black litterman model and calculate corresponding cov matrix and returns
- Find the optimal weights using bl cov matrix and returns

In [49]:
from pypfopt import BlackLittermanModel, plotting
from pypfopt import black_litterman, risk_models

# Fit non linear ML models to predict stock returns into future
apple_returns = (
    (apple_data['Close'] / apple_data['Close'].shift(1))-1).dropna().tolist()
google_returns = (
    (google_data['Close'] / google_data['Close'].shift(1))-1).dropna().tolist()

apple_x = [apple_returns[i:i+10] for i in range(len(apple_returns)-20)]
apple_y = [apple_returns[i+10] for i in range(10,len(apple_returns)-10)]

apple_test = [apple_returns[i:i+10] for i in range(len(apple_returns)-20,len(apple_returns)-10)]

reg = RandomForestRegressor()
reg.fit(apple_x,apple_y)
apple = reg.predict(apple_test)

google_x = [google_returns[i:i+10] for i in range(len(google_returns)-20)]
google_y = [google_returns[i+10] for i in range(10,len(google_returns)-10)]

google_test = [google_returns[i:i+10] for i in range(len(google_returns)-20,len(google_returns)-10)]

reg = RandomForestRegressor()
reg.fit(google_x,google_y)
google = reg.predict(google_test)

future_returns = {'apple':apple,'google':google}
print(apple[-1],google[-1])

# Use predicted future returns as views for BL model
viewdict = {'AAPL': apple[-1], 'GOOGL': google[-1]}
tickers = ['AAPL','GOOGL']
mcaps = {}
for t in tickers:
    stock = yf.Ticker(t)
    mcaps[t] = stock.info["marketCap"]
print(mcaps)

prices = pd.DataFrame({'AAPL':apple_data['Close'].values,'GOOGL':google_data['Close'].values})
S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
market_prices = yf.download("SPY", period="max")["Adj Close"]


delta = black_litterman.market_implied_risk_aversion(market_prices)
print(delta)
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)
print(market_prior)

# Assign confidence measure for stock returns based on some heuristics
confidences = [
    0.6,
    0.4
]

# Fit BL model
bl = BlackLittermanModel(S, pi=market_prior, absolute_views=viewdict, omega="idzorek", view_confidences=confidences)

# Get expected returns
ret_bl = bl.bl_returns()
print(ret_bl)

# Get cov matrix
S_bl = bl.bl_cov()


from pypfopt import EfficientFrontier, objective_functions

ef = EfficientFrontier(ret_bl, S_bl)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()
print(weights)


0.002787829125630311 -0.0006042058258033178
{'AAPL': 2914056404992, 'GOOGL': 1506420850688}
[*********************100%***********************]  1 of 1 completed
2.571631627510369
AAPL     0.271861
GOOGL    0.232587
dtype: float64
AAPL     0.094105
GOOGL    0.085640
dtype: float64
OrderedDict([('AAPL', 0.53258), ('GOOGL', 0.46742)])


/usr/local/lib/python3.10/dist-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


# Algorithmic trading - Using Bollinger Bands (You can use any alternative approach like EMA etc)

- Install talib to calculate Bollinger Bands
- Calculate Bollinger bands for your stock using historical data
- Divide data into independent variable X (stock prices) and dependent variable y (sell -1 if stock price > upper limit of bollinger band, buy 1 otherwise)
- Train a classifier to predict the buy or sell based on stock price
- Use the trained classifier to trade the stock in real time
- The algorithmic trading is more useful in intra day or high frequency trading scenario and is not generally done for long term portfolio holdings
- However, you can buy or sell the complete stock based on this everyday to maximize your capital

Install this package on your system: https://pypi.org/project/TA-Lib/

In [ ]:
import numpy as np
import pandas as pd
import talib
from sklearn.ensemble import RandomForestClassifier


apple_prices = apple_data['Close']

# Calculate Bollinger Bands for Apple
apple_bb_upper, apple_bb_middle, apple_bb_lower = talib.BBANDS(
    apple_prices, timeperiod=20)


# Calculate the Bollinger Bands percentages for Apple
apple_bb_percentage = (apple_prices - apple_bb_lower) / \
    (apple_bb_upper - apple_bb_lower)

# # Combine the Bollinger Bands percentages into a single DataFrame
# bb_percentages = pd.concat(
#     [apple_bb_percentage, google_bb_percentage], axis=1).dropna()

# Define the features and target variable for the machine learning model
# Independent variables (Bollinger Bands percentages for the past period)
X = apple_bb_percentage.values[:-1]
# Target variable (-1 for sell, 1 for buy)
y = np.where(apple_prices.values[1:] > apple_bb_upper[:-1], -1, 1)

# Train a machine learning model using random forest classifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X, y)

# Use the trained model to predict the trading signals for the testing set
# Ideally this will be your real time stock prices
y_pred = model.predict(X_test)

# Perform algorithmic trading based on the predicted signals (example logic)
capital = 100000  # Initial capital in USD
position = 0  # Current position (0 for neutral, 1 for long, -1 for short)

for i in range(len(y_pred)):
    if y_pred[i] == 1 and position != 1:  # Buy signal
        position = 1
        # Place a buy order based on your trading platform's API or logic

    elif y_pred[i] == -1 and position != -1:  # Sell signal
        position = -1
        # Place a sell order based on your trading platform's API or logic

    elif y_pred[i] == 0 and position != 0:  # Exit position
        position = 0
        # Close the existing position based on your trading platform's API or logic

# Calculate the final capital after the trading period
final_capital = capital  # Assume no transaction costs or slippage
# Calculate the final capital based on your trading platform's API or logic

print("Final Capital:", final_capital)
